In [1]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [2]:
#METSIM

In [3]:
METSIM_RESULTS <- read.table("../../Summary_Statistic//Formatted//METSIM_hg19_METAL_POS.TAB",header=F,fill=TRUE,row.names=NULL)
colnames(METSIM_RESULTS) <- c("CHROMOSOME","POSITION","MARKERNAME","METSIM_NEA","METSIM_EA","METSIM_N","PVALUE_FE","METSIM_BETA","METSIM_SE")

In [4]:
METSIM_MAF <- read.table("../../Summary_Statistic//Formatted/CHR_POS_MAF.TAB",header=F,fill=TRUE,row.names=NULL)
colnames(METSIM_MAF) <- c("CHROMOSOME","POSITION","MAF")

In [5]:
  METSIM_DF <- merge(METSIM_RESULTS,METSIM_MAF,by=c("CHROMOSOME","POSITION"))

In [6]:
lead <- read.table("../../ANALYSIS/lead_metsim_3mb_merged.txt",fill=TRUE,header=TRUE)
output_df <- merge(METSIM_DF,lead)%>%select(MARKERNAME,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,METSIM_BETA,METSIM_SE,PVALUE_FE,MAF,METSIM_N)%>%
rename(SNP=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,A1=METSIM_EA,A2=METSIM_NEA,freq=MAF,b=METSIM_BETA,se=METSIM_SE,p=PVALUE_FE,n=METSIM_N)%>%distinct(SNP,.keep_all = TRUE)

In [7]:
df_gwas <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_metsim/gwas/",pattern='.z',full.names = TRUE)
df_gwas <- df_gwas[ !grepl(".bgen.bcor.ld", df_gwas) ]
df_eqtl <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_metsim/eqtl/",pattern='.z',full.names = TRUE)
df_eqtl <- df_eqtl[ !grepl(".bgen.bcor.ld", df_eqtl) ]

In [8]:
for (i in seq_along(df_gwas))
{

# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
    # gwas
s_rsid <- unique((read.table(df_gwas[i],header=T,sep=" "))$rsid)
s_chr <- unique((read.table(df_gwas[i],header=T,sep=" "))$chromosome)
tmp_df <- output_df%>%filter(SNP %in% s_rsid)%>%distinct()
file_name <- paste0(tissue,"_",s_chr,".ma")
write.table(tmp_df,paste0("../../ANALYSIS/SMR/metsim_input//",file_name),sep = "\t",row.names=FALSE,quote=FALSE)
    }


In [ ]:
for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])
file_name <- paste0(chr_q,"_",start_q,"_",end_q,".ma")
tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()%>%select(SNP,A1,A2,freq,b,se,p,n)
write.table(tmp_df,paste0("../../ANALYSIS/SMR/metsim_input/",file_name),sep = "\t",row.names=FALSE,quote=FALSE)
}

In [43]:
## Trans-ancestry 

In [44]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/TRANSANCESTRY_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_THREE <- read.table(gzfile("../data/TRANSANCESTRY_GWAS_ALLELECORRECTED.TAB"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_AGEN","PVALUE_METSIM","NA1","NA2","NA3")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
TRANSANCESTRY_RESULTS <- merge(META_THREE,Metasoft_results,by="MARKERNAME")
TRANSANCESTRY_RESULTS <- TRANSANCESTRY_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(AGEN_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+AGEN_EAF*2*AGEN_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(AGEN_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2,-NA3)

In [45]:
lead <- read.table("../../ANALYSIS/lead_transancestry_3mb.txt",fill=TRUE,header=TRUE)
output_df <- merge(TRANSANCESTRY_RESULTS,lead)%>%select(MARKERNAME,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE,PVALUE_FE,TOTAL_N)%>%
rename(SNP=MARKERNAME,A1=METSIM_EA,A2=METSIM_NEA,freq=MAF_AVG,b=BETA_FE,se=STD_FE,p=PVALUE_FE,n=TOTAL_N)

In [46]:
head(output_df)

,SNP,A1,A2,freq,b,se,p,n
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,rs10000081,T,C,0.33229026,-0.0095197,0.00463699,0.04007360,46381
2,rs10000092,T,C,0.38806186,0.0121143,0.00451048,0.00723546,44685
3,rs1000025,T,C,0.28441548,-0.0111126,0.00485617,0.02211700,46325
4,rs10000252,T,C,0.02623173,0.0314747,0.01459080,0.03099270,36369
5,rs10000268,T,A,0.16447520,-0.0106829,0.00585867,0.06823810,46410
6,rs10000289,A,T,0.03235335,0.0396380,0.01743640,0.02300860,39605


In [47]:
####

In [48]:
df_gwas <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_trans/gwas/",pattern='.z',full.names = TRUE)
df_gwas <- df_gwas[ !grepl(".bgen.bcor.ld", df_gwas) ]
df_eqtl <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_trans/eqtl/",pattern='.z',full.names = TRUE)
df_eqtl <- df_eqtl[ !grepl(".bgen.bcor.ld", df_eqtl) ]

In [49]:
trans_coloc <- read.table("../../ANALYSIS/trans_coloc.txt",fill=TRUE,header=TRUE)

In [50]:
head(trans_coloc)

,Tissue,SNP,chr
,<chr>,<chr>,<int>
1,ArteryTibial,rs6537885,1
2,AdiposeSubcutaneous,rs3795823,1
3,MuscleSkeletal,rs3795823,1
4,AdiposeVisceralOmentum,rs3818797,1
5,ArteryAorta,rs3818797,1
6,HeartAtrialAppendage,rs3818797,1


In [52]:

for (i in seq_along(df_gwas))
{

# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
    # gwas
s_rsid <- unique((read.table(df_gwas[i],header=T,sep=" "))$rsid)
s_chr <- unique((read.table(df_gwas[i],header=T,sep=" "))$chromosome)
tmp_df <- output_df%>%filter(SNP %in% s_rsid)%>%distinct()
file_name <- paste0(tissue,"_",s_chr,".ma")
write.table(tmp_df,paste0("../../ANALYSIS/SMR/trans_IV/",file_name),sep = "\t",row.names=FALSE,quote=FALSE)
    }


Warning message in if (file == "") file <- stdout() else if (is.character(file)) {:
"the condition has length > 1 and only the first element will be used"


ERROR: Error in file(file, ifelse(append, "a", "w")): invalid 'description' argument


In [ ]:
###

In [9]:
for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])
file_name <- paste0(chr_q,"_",start_q,"_",end_q,".ma")
tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()%>%select(SNP,A1,A2,freq,b,se,p,n)
write.table(tmp_df,paste0("../../ANALYSIS/SMR/trans_input/",file_name),sep = "\t",row.names=FALSE,quote=FALSE)
}

In [ ]:
### European 

In [13]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_TWO <- read.table(gzfile("../data/EUROPEAN_GWAS_ALLELECORRECTED.TAB"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)

In [14]:
lead <- read.table("../../ANALYSIS/lead_european_3mb.txt",fill=TRUE,header=TRUE)
 output_df <- merge(EUR_RESULTS,lead)%>%select(MARKERNAME,METSIM_EA,METSIM_NEA,MAF_AVG,BETA_FE,STD_FE,PVALUE_FE,TOTAL_N)%>%
rename(SNP=MARKERNAME,A1=METSIM_EA,A2=METSIM_NEA,freq=MAF_AVG,b=BETA_FE,se=STD_FE,p=PVALUE_FE,n=TOTAL_N)

In [15]:
df_gwas <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/gwas/",pattern='.z',full.names = TRUE)
df_gwas <- df_gwas[ !grepl(".bgen.bcor.ld", df_gwas) ]
df_eqtl <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/eqtl/",pattern='.z',full.names = TRUE)
df_eqtl <- df_eqtl[ !grepl(".bgen.bcor.ld", df_eqtl) ]

In [16]:

for (i in seq_along(df_gwas))
{

# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
    # gwas
s_rsid <- unique((read.table(df_gwas[i],header=T,sep=" "))$rsid)
s_chr <- unique((read.table(df_gwas[i],header=T,sep=" "))$chromosome)
tmp_df <- output_df%>%filter(SNP %in% s_rsid)%>%distinct()
file_name <- paste0(tissue,"_",s_chr,".ma")
write.table(tmp_df,paste0("../../ANALYSIS/SMR/eur_IV//",file_name),sep = "\t",row.names=FALSE,quote=FALSE)
    }

In [47]:
###

In [14]:
for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])
file_name <- paste0(chr_q,"_",start_q,"_",end_q,".ma")
tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()%>%select(SNP,A1,A2,freq,b,se,p,n)
write.table(tmp_df,paste0("../../ANALYSIS/SMR/eur_input/",file_name),sep = "\t",row.names=FALSE,quote=FALSE)
}